Projeto de Transfer Learning para classificar personagens do Naruto

Iremos usar o VGG16 para classificar imagens dos personagens de Naruto. 

As imagens foram obtidas do repositório: https://github.com/Osunadev/naruto-character-classification. Lá existem 300 imagens divididas em 10 personagens.

Primeiro vamos carregar o VGG16 e remover sua camada final, a camada de classificação softmax de 1000 classes específica para ImageNet, e substituí-la por uma nova camada de classificação para as classes sobre as quais estamos treinando. Em seguida, congelaremos todos os pesos na rede, exceto os novos que se conectam à nova camada de classificação, e treinaremos a nova camada de classificação sobre nosso novo conjunto de dados.

In [ ]:
%matplotlib inline

import os

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

Tensor-flow backend

In [ ]:
hroot = 'Naruto_images'

train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

Carregamento do banco de imagens do Naruto

In [ ]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

A função acima é para pré-processar a imagem

In [ ]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

Carrega todas as imagens da pasta root

In [ ]:
random.shuffle(data)

Randomiza a ordem dos dados

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

Cria training / validation / test split (70%, 15%, 15%)

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

Separa as dados das labels

In [ ]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

Pré-processamento dos dados como antes, certificando-se de que sejam float32 e normalizados entre 0 e 1.

In [ ]:

vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

Carremento do VGG16 do Keras e uma impressão do sumário. Note o tamanho da rede neural.

In [ ]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)

Aqui aplicamos a mudança no VGG16 que dizemos que ia aplicar.

In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Vamos treinar novamente essa rede, model_new no novo conjunto de dados e rótulos. Mas primeiro, precisamos congelar os pesos e desvios em todas as camadas da rede, exceto nossa nova no final, com a expectativa de que os recursos aprendidos no VGG ainda sejam bastante relevantes para a nova tarefa de classificação de imagens.

In [ ]:

history2 = model_new.fit(x_train, y_train, 
                         batch_size=128, 
                         epochs=10, 
                         validation_data=(x_val, y_val))

Como antes, vamos em frente e treinamos o novo modelo, usando os mesmos hiperparâmetros (tamanho do lote e número de épocas) de antes, juntamente com o mesmo algoritmo de otimização. Também acompanhamos sua história à medida que avançamos.

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Impressão da perda e da acurácia do modelo.

In [ ]:
img, x = get_image('Naruto_images/Jiraya/image07.jpeg')
probabilities = model_new.predict([x])

Para prever uma nova imagem, basta executar o código a seguir para obter as probabilidades de cada classe.